In [ ]:
!kaleido_get_chrome

In [ ]:
import os
import pickle

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from experiments import Experiment, ExperimentArray
from tqdm.auto import tqdm

np.random.seed(1)

pio.templates["plotly_white_custom"] = pio.templates["plotly_white"]
pio.templates["plotly_white_custom"].update({"layout_font_size": 22})
pio.templates.default = "plotly_white_custom"

group_cols = ["d", "# Hidden", "Width"]

In [ ]:
def format_mean_std(group, exp_threshold=4):
    mean = group.mean()
    std = group.std() or 0
    if np.isnan(mean):
        return None
    if mean == float("inf") or std == float("inf"):
        return "\\infty"
    mean_exponent = int(np.floor(np.log10(abs(mean)))) if mean != 0 else 0
    mean_mantissa = mean / (10**mean_exponent)
    if std == 0:
        std_exponent = 0
    elif np.isnan(std):
        std_exponent = None
    else:
        std_exponent = int(np.floor(np.log10(abs(std))))
    std_mantissa = std / (10**std_exponent) if std_exponent else 0
    mean_str = f"{mean:.2f}" if mean_exponent < exp_threshold else f"{mean_mantissa:.2f}$\\times10^{mean_exponent}$"
    std_str = (
        (f"$\\pm${std:.2f}" if std_exponent < exp_threshold else f"$\\pm${std_mantissa:.2f}$\\times10^{std_exponent}$")
        if std_exponent is not None
        else ""
    )
    return mean_str + std_str

In [ ]:
# log_dirs += ["../results/run_15", "../results/run_16"]
log_dirs = ["../results/"]
df = pd.DataFrame()
for log_dir in log_dirs:
    count_dir = os.path.join(log_dir, "shi_counts")
    print("Loading Results From:", count_dir)
    if os.path.isdir(count_dir):
        new_df = pd.DataFrame(
            [
                pickle.load(open(os.path.join(count_dir, f), "rb"))
                for f in tqdm(os.listdir(count_dir), desc="Loading Results")
            ]
        )
    else:
        raise FileNotFoundError(f"Directory {count_dir} not found")
    df = pd.concat([df, new_df], ignore_index=True)

print("Number of Results:", len(df))

In [ ]:
df["d"] = df["Layer Widths"].apply(lambda w: w[0])
df["Width"] = df["Layer Widths"].apply(lambda w: w[1])
df["# Hidden"] = df["Layer Widths"].apply(lambda x: len(x) - 2)
df["# Relus"] = df["Layer Widths"].apply(lambda x: sum(x[1:-1]))
df["Average Volume"] = df["Volumes"].map(
    lambda x: sum(y for y in x if y is not None and y <= 10000) / len(x), na_action="ignore"
)
df["Volume > 10000"] = df["Volumes"].map(
    lambda x: sum(y > 10000 for y in x if y is not None)
).fillna(0)
df["Average Inradius"] = df["Inradii"].map(
    lambda x: sum(y for y in x if y is not None and y <= 10000) / len(x), na_action="ignore"
)
df["Average Polys / Second"] = df["# Regions"] / df["Search Time"]
df["Hours"] = df["Search Time"] / 3600
df["% Finite"] = df["Inradii"].map(
    lambda x: sum(y is not None and y != float("inf") for y in x) / len(x), na_action="ignore"
)


In [ ]:
df.columns

In [ ]:
df["Diameter"] = (df["Diameter UB"] + df["Diameter LB"]) / 2

In [ ]:
result = (
    df[group_cols + ["# Regions", "Avg # Facets", "Average Volume", "% Finite", "Diameter"]]
    .groupby(group_cols)
    .agg(lambda x: format_mean_std(x))
)

os.makedirs("../figures", exist_ok=True)

with open("../figures/search_summary_full.tex", "w") as f:
    f.write(result.to_latex().replace("#", "\\#").replace("%", "\\%"))
# breakpoint()
with open("../figures/search_summary_half.tex", "w") as f:
    f.write(
        result[["# Regions", "Avg # Facets", "Diameter"]].loc[4:].to_latex().replace("#", "\\#").replace("%", "\\%")
    )

In [ ]:
hist = (
    df.explode("Facet Counts")
    .reset_index()
    .value_counts(subset=["d", "Width", "# Hidden", "Facet Counts", "Trial"], sort=False)
    .sort_index()
)  # .reset_index()
hist = hist[hist > 1]
hist = hist.groupby(["d", "Width", "# Hidden", "Facet Counts"]).agg(["mean", "std"]).reset_index()  # .sort_index()
hist


hist[hist["Facet Counts"] < hist["d"]]


hist = hist[(hist["Facet Counts"] >= hist["d"]) | (hist["d"] >= hist["# Hidden"] * hist["Width"])]


font = dict(
    size=34,
    family="Times New Roman",
)


hist["# Hidden"] = hist["# Hidden"].astype(str)
hist = hist[hist["mean"] != 1]
hist = hist[hist["std"].notna()]
facet_col = "Width"
color = "# Hidden"
facet_row = "d"

fig = px.bar(
    hist,
    x="Facet Counts",
    y="mean",
    facet_col=facet_col,
    color=color,
    facet_row=facet_row,
    log_y=True,
    barmode="stack",
    error_y="std",
    template="plotly_white",
    category_orders={"d": sorted(hist["d"].unique()), "# Hidden": sorted(hist["# Hidden"].unique())},
    labels={
        "mean": "Number of Polyhedrons",
        "Facet Counts": "Number of Neighbors",
        "d": "Dimension",
        "# Hidden": "# Hidden Layers",
    },
    title=" ",
    facet_row_spacing=0.01,
    facet_col_spacing=0,
)
fig.update_layout(font=font)
fig.update_layout(
    showlegend=True, width=len(hist[facet_col].unique()) * 400 + 100, height=len(hist[facet_row].unique()) * 300
)
# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1], font=font))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("=", ": "), font=font))
fig.for_each_yaxis(lambda y: y.update(title=" "))
fig.for_each_xaxis(lambda x: x.update(title=" "))
fig.add_annotation(
    x=-0.03,
    y=0.5,
    text="Number of Polyhedrons",
    textangle=-90,
    xref="paper",
    yref="paper",
    font=font | {"size": 38},
    showarrow=False,
)
fig.add_annotation(
    x=0.5, y=-0.08, text="Number of Neighbors", xref="paper", yref="paper", font=font | {"size": 38}, showarrow=False
)

# fig.update_layout(legend=dict(
#     font=font|{"size": 20},
#     traceorder="reversed",
# ))

fig.update_layout(
    legend=dict(
        font=font | {"size": 26},
        orientation="h",
        yanchor="bottom",
        y=1.04,
        xanchor="left",
        x=0,
    )
)

global_max = hist["Facet Counts"].max()
max_per_col = hist.groupby(facet_col)["Facet Counts"].max().tolist()

max_facets = hist["Facet Counts"].max()
for j, d in enumerate(sorted(hist[facet_row].unique())):
    row = len(hist[facet_row].unique()) - j
    for i, w in enumerate(sorted(hist[facet_col].unique())):
        col = i + 1

        max_y = (
            hist.groupby([facet_row, facet_col, "Facet Counts"])
            .agg({"mean": "sum"})
            .reset_index()
            .groupby([facet_row, facet_col])
            .agg({"mean": "max"})
            .loc[d, w]
            .item()
            * 5
        )

        for c in hist[color].unique():
            rows = hist[(hist[facet_row] == d) & (hist[facet_col] == w) & (hist[color] == c)]
            if len(rows) == 0:
                continue

            # marker_style = {
            #     "symbol": "arrow",
            #     "size": 15,
            #     "angle": 180,
            # }
            marker_style = {
                "symbol": "diamond-tall",
                "size": 15,
                "angle": 0,
            }

            mean = ((rows["Facet Counts"].values @ rows["mean"].values) / rows["mean"].sum()).item()
            # fig2 = px.scatter(x=[mean], y=[max_y])
            # fig2.update_traces(
            #     marker=dict(color=px.colors.qualitative.Plotly[int(c) - 1], opacity=0.7, **marker_style),
            #     showlegend=False,
            # )
            # for a in fig2.data:
            #     fig.add_trace(a, row=row, col=col)

            # fig2 = px.scatter(x=[mean], y=[max_y], text=[f"{mean:.2f}"])
            # fig2.update_traces(marker=dict(symbol="arrow", size=15, color=px.colors.qualitative.Plotly[int(c)-1], opacity=0.5, angle=180), textposition='middle right', showlegend=False, textfont=font|{"size": 16})
            # for a in fig2.data:
            #     fig.add_trace(a, row=row, col=col)

            # fig2 = px.scatter(x=[max_per_col[i]-1], y=[10**(int(c)*1+2)], text=[f"{mean:.2f}"])
            # fig2.update_traces(marker=dict(color=px.colors.qualitative.Plotly[int(c)-1], opacity=1, **marker_style), textposition='middle left', showlegend=False, textfont=font|{"size": 16})
            fig2 = px.scatter(x=[min(max_per_col[i] + 1, global_max)], y=[11 ** (int(c) * 1 + 2)], text=[f"{mean:.2f}"])
            fig2.update_traces(
                marker=dict(color=px.colors.qualitative.Plotly[int(c) - 1], opacity=1, **marker_style),
                textposition="middle left",
                showlegend=False,
                textfont=font | {"size": 30},
            )
            for a in fig2.data:
                fig.add_trace(a, row=row, col=col)

            # fig3 = px.scatter(x=[2*d], y=[max_y])
            # fig3.update_traces(marker=dict(symbol="triangle-right", size=15, color="DarkSlateGrey"), line=dict(width=4, color="DarkSlateGrey"), textposition='top center', showlegend=False)
            # for a in fig3.data:
            #     fig.add_trace(a, row=row, col=col)

### Resize plots based on max face count
total = sum(max_per_col)
relative_widths = [(m / total) - 0.1 / len(max_per_col) for m in max_per_col]

positions = []
current = 0.07
for width in relative_widths:
    end = current + width
    positions.append((current, end))
    current = end
positions = positions * len(hist[facet_row].unique())

fig.update_xaxes(matches=None)
for idx, (start, end) in enumerate(positions):
    xaxis_name = "xaxis" if idx == 0 else f"xaxis{idx + 1}"
    fig.update_layout(
        {
            xaxis_name: {
                "domain": [start, end],
                "range": [1, max_per_col[idx % len(max_per_col)] + 2],
            }
        }
    )

fig.show()
fig.write_image("../figures/Poly Counts Full.svg")

In [ ]:
font  = dict(
    # family="Courier New, monospace",
    size=34,
    family="Times New Roman",
    color="black",
)

df["Diameter"] = (df["Diameter UB"] + df["Diameter LB"]) / 2
df["dub"] = df["Width"] ** df["# Hidden"]
df["dubl"] = np.log(df["# Regions"])/np.log(df["Width"] * df["# Hidden"])
# df["d"] = df["d"].astype(int)
df

In [ ]:
x = "dub"
xaxis_type = "log"
xaxis_title = "Width<sup>Depth</sup>"

try:
    df = df.reset_index()
except:
    pass
fig = go.Figure()
outer = df.groupby(["d"]+[x]).agg({"Diameter LB": "min", "Diameter UB": "max", "Diameter": "mean"}).reset_index().set_index("d")
outer.sort_values([x, "Diameter"], inplace=True, ignore_index=False)
for d in df["d"].unique():
    fig.add_trace(go.Scatter(x=outer.loc[d, x], y=outer.loc[d, "Diameter LB"],
        fill=None,
        mode='lines',
        line_dash="dash",
        line_color=px.colors.sequential.Plasma[2*d-1],
        legendgroup=str(d),
        showlegend=False,
        name=f"{d}",
        hoverinfo="x+text",
        # text=outer.loc[d, "Width"].astype(int).astype(str) + " Width , " + outer.loc[d, "# Hidden"].astype(int).astype(str) + " Depth"
    ))
    fig.add_trace(go.Scatter(x=outer.loc[d, x], y=outer.loc[d, "Diameter UB"],
        fill=None,
        mode='lines',
        line_dash="dash",
        line_color=px.colors.sequential.Plasma[2*d-1],
        legendgroup=str(d),
        showlegend=False,
        name=f"{d}",
        hoverinfo="x+text",
        # text=outer.loc[d, "Width"].astype(int).astype(str) + " Width , " + outer.loc[d, "# Hidden"].astype(int).astype(str) + " Depth"
    ))
for d in df["d"].unique():
    fig.add_trace(go.Scatter(x=outer.loc[d,x], y=outer.loc[d, "Diameter"],
        mode='markers+lines',
        line_color=px.colors.sequential.Plasma[2*d-1],
        legendgroup=str(d),
        name=str(d),
        hoverinfo="x+text",
        # text=outer.loc[d, "Width"].astype(int).astype(str) + " Width , " + outer.loc[d, "# Hidden"].astype(int).astype(str) + " Depth"
    ))

# fig.add_trace(go.Scatter(x=df["dub"], y=df["Diameter"],
#     mode='markers',
#     marker_color=[px.colors.sequential.Plasma[2*d-1] for d in df["d"]],))

fig.update_traces(marker_size=12, line=dict(width=8))


fig.update_layout(
    font=font,
    yaxis_title="Dual Graph Diameter",
    xaxis_title=xaxis_title,
    xaxis_type=xaxis_type,
    width=800,
    height=1400,
    legend_title_text="<i>d</i>",
    xaxis_dtick=1
)
fig.show()
fig.write_image("../figures/Diameter vs Upper Bound Together.svg")

In [ ]:
x = "dubl"
xaxis_type = "linear"
xaxis_title = "Lower Bound" # "Width<sup>Depth</sup>"

try:
    df = df.reset_index()
except:
    pass
fig = go.Figure()
outer = df.groupby(["d"]+[x]).agg({"Diameter LB": "min", "Diameter UB": "max", "Diameter": "mean"}).reset_index().set_index("d")
outer.sort_values([x, "Diameter"], inplace=True, ignore_index=False)
for d in df["d"].unique():
    fig.add_trace(go.Scatter(x=outer.loc[d, x], y=outer.loc[d, "Diameter LB"],
        fill=None,
        mode='lines',
        line_dash="dash",
        line_color=px.colors.sequential.Plasma[2*d-1],
        legendgroup=str(d),
        showlegend=False,
        name=f"{d}",
        hoverinfo="x+text",
        # text=outer.loc[d, "Width"].astype(int).astype(str) + " Width , " + outer.loc[d, "# Hidden"].astype(int).astype(str) + " Depth"
    ))
    fig.add_trace(go.Scatter(x=outer.loc[d, x], y=outer.loc[d, "Diameter UB"],
        fill=None,
        mode='lines',
        line_dash="dash",
        line_color=px.colors.sequential.Plasma[2*d-1],
        legendgroup=str(d),
        showlegend=False,
        name=f"{d}",
        hoverinfo="x+text",
        # text=outer.loc[d, "Width"].astype(int).astype(str) + " Width , " + outer.loc[d, "# Hidden"].astype(int).astype(str) + " Depth"
    ))
for d in df["d"].unique():
    fig.add_trace(go.Scatter(x=outer.loc[d,x], y=outer.loc[d, "Diameter"],
        mode='markers+lines',
        line_color=px.colors.sequential.Plasma[2*d-1],
        legendgroup=str(d),
        name=str(d),
        hoverinfo="x+text",
        # text=outer.loc[d, "Width"].astype(int).astype(str) + " Width , " + outer.loc[d, "# Hidden"].astype(int).astype(str) + " Depth"
    ))

# fig.add_trace(go.Scatter(x=df["dub"], y=df["Diameter"],
#     mode='markers',
#     marker_color=[px.colors.sequential.Plasma[2*d-1] for d in df["d"]],))

fig.update_traces(marker_size=12, line=dict(width=4))


fig.update_layout(
    font=font,
    yaxis_title="Dual Graph Diameter",
    xaxis_title=xaxis_title,
    xaxis_type="log",
    width=1400,
    height=600,
    legend_title_text="<i>d</i>",
)
fig.show()
fig.write_image("../figures/Diameter vs Lower Bound Together.svg")

In [ ]:
df = df.reset_index(drop=None in df.index.names)
facet_col = "Width"
outer = df.groupby(["d", facet_col]+["dub"]).agg({"Diameter LB": "min", "Diameter UB": "max", "Diameter": "mean", "# Regions": "max"}).reset_index().sort_values("dub")
# px.line(outer, x="dub", y="# Regions", color="d", width=800, height=800, log_y=True, log_x=True).show()
fig = px.scatter(df, x="dub", y="# Regions", color="d", width=800, height=800, log_y=True, log_x=True)
fig.update_traces(marker_size=10, opacity=0.8, line=dict(width=3))
fig.update_layout(
    xaxis_title="Width<sup>Depth</sup>",
)

In [ ]:
try:
    df = df.reset_index()
except:
    pass
facet_col = "Width"
outer = df.groupby(["d", facet_col]+["dub"]).agg({"Diameter LB": "min", "Diameter UB": "max", "Diameter": "mean"}).reset_index().set_index("d")
outer.sort_values("dub", inplace=True, ignore_index=False)
df = df.set_index("d")
for w in df[facet_col].unique():
    print(f"{facet_col}: {w}")
    fig = go.Figure()
    for d in df[df[facet_col] == w].index.unique():
        fig.add_trace(go.Scatter(x=outer[outer[facet_col] == w].loc[d, "dub"], y=outer[outer[facet_col] == w].loc[d, "Diameter LB"],
            fill=None,
            mode='lines',
            line_dash="dash",
            line_color=px.colors.sequential.Plasma[2*d-1],
            line_width=6,
            legendgroup=str(d),
            showlegend=False,
            name=f"{d}",
            hoverinfo="x+text",
            # text=outer.loc[d, "Width"].astype(int).astype(str) + " Width , " + outer.loc[d, "# Hidden"].astype(int).astype(str) + " Depth"
        ))
        fig.add_trace(go.Scatter(x=outer[outer[facet_col] == w].loc[d, "dub"], y=outer[outer[facet_col] == w].loc[d, "Diameter UB"],
            fill=None,
            mode='lines',
            line_dash="dash",
            line_color=px.colors.sequential.Plasma[2*d-1],
            line_width=6,
            legendgroup=str(d),
            showlegend=False,
            name=f"{d}",
            hoverinfo="x+text",
        ))
    for d in df.index.unique():
        fig.add_trace(go.Scatter(x=outer[outer[facet_col] == w].loc[d,"dub"], y=outer[outer[facet_col] == w].loc[d, "Diameter"],
            mode='lines',
            line_color=px.colors.sequential.Plasma[2*d-1],
            legendgroup=str(d),
            line_width=6,
            name=str(d),
            hoverinfo="x+text",
        ))
        # fig.add_trace(go.Scatter(x=df[df[facet_col] == w].loc[d, "dub"], y=df[df[facet_col] == w].loc[d, "Diameter LB"],
        #     fill=None,
        #     mode='markers',
        #     # line_dash="dash",
        #     line_color=px.colors.sequential.Plasma[2*d-1],
        #     legendgroup=str(d),
        #     showlegend=False,
        #     name=f"{d}",
        #     hoverinfo="x+text",
        #     opacity=0.5
        # ))
        # fig.add_trace(go.Scatter(x=df[df[facet_col] == w].loc[d, "dub"], y=df[df[facet_col] == w].loc[d, "Diameter UB"],
        #     fill=None,
        #     mode='markers',
        #     # line_dash="dash",
        #     line_color=px.colors.sequential.Plasma[2*d-1],
        #     legendgroup=str(d),
        #     showlegend=False,
        #     name=f"{d}",
        #     hoverinfo="x+text",
        #     opacity=0.5
        #     # text=outer.loc[d, "Width"].astype(int).astype(str) + " Width , " + outer.loc[d, "# Hidden"].astype(int).astype(str) + " Depth"
        # ))

    fig.update_traces(marker_size=6, line=dict(width=8))


    fig.update_layout(
        font=font,
        yaxis_title="Dual Graph Diameter",
        xaxis_title="Width<sup>Depth</sup>",
        xaxis_type="log",
        width=800,
        height=800,
        legend_title_text="<i>d</i>",
        xaxis_dtick=1
    )
    fig.show()
    fig.write_image(f"../figures/Diameter vs Bound {facet_col} {w}.svg")
df = df.reset_index()

In [ ]:
df.groupby(["d"]+["dub", "Width"]).size()

In [ ]:
fig = go.Figure()
outer = df.groupby(["d"]+["dub"]).agg({"Diameter LB": "min", "Diameter UB": "max", "Diameter": "mean"}).reset_index().set_index("d")
outer.sort_values("dub", inplace=True, ignore_index=False)
for d in df["d"].unique():
    rgb = tuple(int(px.colors.sequential.Plasma[2*d-1][i:i+2], 16) for i in (1, 3, 5))
    fillcolor = f"rgba({rgb[0]}, {rgb[1]}, {rgb[2]}, 0.3)"
    fig.add_trace(go.Scatter(x=outer.loc[d, "dub"], y=outer.loc[d, "Diameter LB"],
        fill=None,
        mode='lines',
        # line_color=fillcolor,
        line_color=px.colors.sequential.Plasma[2*d-1],
        legendgroup=str(d),
        # line_dash="dash",
        showlegend=False,
        name=f"{d}",
        hoverinfo="x+text",
        # text=outer.loc[d, "Width"].astype(int).astype(str) + " Width , " + outer.loc[d, "# Hidden"].astype(int).astype(str) + " Depth"
    ))
    fig.add_trace(go.Scatter(x=outer.loc[d, "dub"], y=outer.loc[d, "Diameter UB"],
        fill="tonexty",
        mode='lines',
        line_color=fillcolor,
        fillcolor=fillcolor,
        legendgroup=str(d),
        # line_dash="dash",
        showlegend=False,
        name=f"{d}",
        hoverinfo="x+text",
        # text=outer.loc[d, "Width"].astype(int).astype(str) + " Width , " + outer.loc[d, "# Hidden"].astype(int).astype(str) + " Depth"
    ))
for d in df["d"].unique():
    fig.add_trace(go.Scatter(x=outer.loc[d,"dub"], y=outer.loc[d, "Diameter"],
        mode='lines',
        line_color=px.colors.sequential.Plasma[2*d-1],
        legendgroup=str(d),
        line_dash="dash",
        line_width=5,
        showlegend=True,
        name=f"{d}",
    ))

fig.update_traces(marker_size=6, line=dict(width=6))


fig.update_layout(
    font=font,
    yaxis_title="Dual Graph Diameter",
    xaxis_title="Width<sup>Depth</sup>",
    xaxis_type="log",
    width=800,
    height=800,
    legend_title_text="<i>d</i>",
    xaxis_dtick=1
)
fig.show()
fig.write_image("../figures/Diameter vs Bound.svg")